<a href="https://colab.research.google.com/github/TottiPuc/Machine_learning/blob/master/Regresi%C3%B3n_logistica_Redes_Neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión Logistica para Clasificación de Imágenes

En un notebook anterior "[clasificación con Keras](https://github.com/TottiPuc/Machine_learning/blob/master/Clasificaci%C3%B3n_NN_feedforward_Keras.ipynb)" fue  implementado un modelo de calsificación de imágenes usando KERAS una herramienta muy poderosa que hace parte de tensorflow y que nos facilita la vida en terminos de programación.

En este notebook se implementara paso  a paso el algoritmo de regresión logistica el cual tiene como objetivo buscar la probabilidad de que un dado elemento pertenezca a una determinada clase, ( a manera informativa esta etapa de regresión logistica es la que se implementa al final de caulquier tipo de red neuronal sea normal o de profundidad DEEP)  

El objetivo es mostrar cada una de las etapas de la red neuronal y todos los pasos matematicos para una mejor comprehension los cuales son:



*   Inicialización de pesos
*   Cálculo del error entre la imágen original y la estimada
*   Cálculo de la derivada de la funcion de error
*   Entrenamiento y evaluación del modelo implementado



Primero importamos nuestras librerias correspondientes

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import math
from tensorflow.keras.datasets import fashion_mnist
import pandas as pd
from tqdm import tqdm_notebook

Como primera medida importamos y hacemos el procesamiento adecuado del dataset fashion MNIST disponible en keras y publicado en [fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)

##Cargando datos

In [3]:
(x_entrenamiento, y_entrenamiento),(x_teste,y_teste) = fashion_mnist.load_data()
#Normalizamos las imagenes para que qeuden en el rango entre 0 y 1 el valor de cada pixel
x_entrenamiento, x_teste = x_entrenamiento/255.0 , x_teste/255.0

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
print("dimensiones de los datos de entrenamiento ", x_entrenamiento.shape)
print("dimensiones de los atiquetas de entrenamiento ", y_entrenamiento.shape)
print("dimensiones de los datos de teste ", x_teste.shape)

dimensiones de los datos de entrenamiento  (60000, 28, 28)
dimensiones de los atiquetas de entrenamiento  (60000,)
dimensiones de los datos de teste  (10000, 28, 28)


Como la entrada de toda red neuronal osea su primera capa de neurones recibe cada matriz de pixeles, es necesario convertir la matriz en un vector.
En nuestro caso cada matriz es de 28x28 lo que haremos es convertir esta entrada en un vector de 1x784

In [0]:
x_entrenamiento = tf.reshape(x_entrenamiento,shape=(-1,784))
x_teste = tf.reshape(x_teste,shape=(-1,784))

##Creando el modelo

Lo primero que se hace en todo tipo de modelo es iniciar los pesos del modelo para que no inicien en cero que no es recomendable, y se deben iniciar con valores muy pequeños

In [0]:
pesos = tf.Variable(tf.random.normal(shape =(784,10), dtype=tf.float64)) # se da esta dmensión para que al multiplicar cada entrada con los pesos cada valor
                                                                                                # del producto punto tenga un valor diferente, si no todos los pesos serian iguales
bias = tf.Variable(tf.random.normal(shape = (10,), dtype=tf.float64))


Creamos la función de regresion logistica

In [0]:
def logistic_regression(x):
    lr = tf.add(tf.matmul(x, pesos), bias)
    return lr

Creamos la funcion de coste que serà nuestra funcion que minimice los errores en este caso usaremos la funcion de entropia cruzada

In [0]:
def cross_entropy(y_real, y_pred):
    y_real = tf.one_hot(y_real, 10)
    perdidas = tf.nn.softmax_cross_entropy_with_logits(labels = y_real, logits = y_pred)
    return tf.reduce_mean(perdidas)



Optimizamos el entrenamiento creando la función del Gradiente

In [0]:
def gradiente(x, y):
    with tf.GradientTape() as tape:
        y_pred = logistic_regression(x)
        loss_val = cross_entropy(y, y_pred)
    return tape.gradient(loss_val, [pesos, bias])



#Entrenando el modelo

Creamos los hiperparametros de nuestro modelo

In [0]:
numero_de_lotes = 10000
tasa_aprendizaje=0.01
tamaño_lote= 128

Lo importante en toda etapa de optimización es saber que dirección y si se debe aumentar o disminuir el valor del gradiente a los pesos, para eso se utiliza una funcion de optimizacion del gradiente estocastico

In [0]:
optimizador = tf.optimizers.SGD(tasa_aprendizaje)

Dejamos preparado el calculo de los aciertos del modelo

In [0]:
def aciertos(y_real, y_predicho):
  y_real = tf.cast(y_real, dtype= tf.int32)
  predicciones = tf.cast(tf.argmax(y_predicho, axis=1),dtype=tf.int32)
  predicciones = tf.equal(y_real, predicciones)
  return tf.reduce_mean(tf.cast(predicciones,dtype=tf.float32))

Un punto importante es saber que como estamos usando el SGD (stochastic gradient dsscent) para optimizar nuestro modelo, no podemos usar el conjunto de entrenamiento completo, primero por que seria una mala practica y segundo por que este metodo esta diseñado para actuar de modo offline es decir con datos de entrada recientes sin necesidad de entrenar todo el modelo cuadno lleguen datos nuevos.

Por eso dividiremos en lotes maspequeños nuestros datos de entrenamiento

In [0]:
dataset_entrenamiento = tf.data.Dataset.from_tensor_slices((x_entrenamiento, y_entrenamiento))
dataset_entrenamiento = dataset_entrenamiento.repeat().shuffle(x_entrenamiento.shape[0]).batch(tamaño_lote)


Ciclo de entrenamiento

In [14]:
for indice_lote, (lote_x_entrenamiento, lote_y_entrenamiento) in enumerate(dataset_entrenamiento.take(numero_de_lotes), 1):

  #calculo del gradientes
  gradientes = gradiente(lote_x_entrenamiento,lote_y_entrenamiento)

  # Otimizamos los pesos con el valor del gradiente
  optimizador.apply_gradients(zip(gradientes, [pesos, bias]))

  #Realizamos una predicción
  y_pred = logistic_regression(lote_x_entrenamiento)

  #calculamos el error
  error = cross_entropy(lote_y_entrenamiento, y_pred)

  #calculamos los aciertos
  acierto = aciertos(lote_y_entrenamiento, y_pred)

  # Print
  print("Número del lote: {}, Error del Modelo: {}, Aciertos en entrenamiento: {}".format (indice_lote, error, acierto))

Streaming output truncated to the last 5000 lines.
Número del lote: 5002, Error del Modelo: 1.7418909483745806, Aciertos en entrenamiento: 0.6015625
Número del lote: 5003, Error del Modelo: 1.4290754099479306, Aciertos en entrenamiento: 0.7109375
Número del lote: 5004, Error del Modelo: 1.261584507148021, Aciertos en entrenamiento: 0.71875
Número del lote: 5005, Error del Modelo: 1.7532305878753105, Aciertos en entrenamiento: 0.6171875
Número del lote: 5006, Error del Modelo: 1.3504546438464968, Aciertos en entrenamiento: 0.71875
Número del lote: 5007, Error del Modelo: 1.6758544462393146, Aciertos en entrenamiento: 0.6328125
Número del lote: 5008, Error del Modelo: 1.8062643944130865, Aciertos en entrenamiento: 0.671875
Número del lote: 5009, Error del Modelo: 1.6956248852969116, Aciertos en entrenamiento: 0.65625
Número del lote: 5010, Error del Modelo: 1.7815195812404396, Aciertos en entrenamiento: 0.640625
Número del lote: 5011, Error del Modelo: 1.6279522976296925, Aciertos en ent

## Provando el modelo

Para saber si nuestro modelo quedo bien entrenado, hacemos los calculos con los datos de prueba

In [0]:
dataset_teste = tf.data.Dataset.from_tensor_slices((x_teste,y_teste))
dataset_teste = dataset_teste.repeat().shuffle(x_teste.shape[0]).batch(tamaño_lote)

Realizamos las predicciones y los calculos de acierto

In [17]:
print("iniciando la evaluación de los testes, espere....")
for indice_lote,(lote_x_teste, lote_y_teste) in enumerate(dataset_teste.take(numero_de_lotes),1):
  y_pred = logistic_regression(lote_x_teste)
  acc = aciertos(lote_y_teste, y_pred)
  acierto = tf.reduce_mean(tf.cast(acc,tf.float64))*100
  
print("La tasa de acierto en teste {} %".format(acierto))

iniciando la evaluación de los testes, espere....
La tasa de acierto en teste 76.5625 %


Realizamos las pruebas con tres imàgenes aleatorias

In [0]:
dataset_teste = tf.data.Dataset.from_tensor_slices((x_teste,y_teste))
dataset_teste = dataset_teste.repeat().shuffle(x_teste.shape[0]).batch(3)

In [19]:
for indice_lote, (lote_xs, lote_ys) in enumerate(dataset_teste.take(3), 1):
    print("\nClase verdadera:", lote_ys)
    y_pred = tf.math.argmax(logistic_regression(lote_xs), axis = 1)
    print("Classe Predicha:", y_pred)




Clase verdadera: tf.Tensor([2 2 5], shape=(3,), dtype=uint8)
Classe Predicha: tf.Tensor([2 6 8], shape=(3,), dtype=int64)

Clase verdadera: tf.Tensor([3 6 8], shape=(3,), dtype=uint8)
Classe Predicha: tf.Tensor([1 2 8], shape=(3,), dtype=int64)

Clase verdadera: tf.Tensor([8 3 5], shape=(3,), dtype=uint8)
Classe Predicha: tf.Tensor([8 3 5], shape=(3,), dtype=int64)
